<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#FeatureProducer.py" data-toc-modified-id="FeatureProducer.py-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>FeatureProducer.py</a></span></li><li><span><a href="#main" data-toc-modified-id="main-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>main</a></span><ul class="toc-item"><li><span><a href="#init" data-toc-modified-id="init-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>init</a></span></li><li><span><a href="#load" data-toc-modified-id="load-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>load</a></span></li><li><span><a href="#preprocess" data-toc-modified-id="preprocess-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>preprocess</a></span></li><li><span><a href="#feature-engineering" data-toc-modified-id="feature-engineering-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>feature engineering</a></span></li><li><span><a href="#fit-and-transform" data-toc-modified-id="fit-and-transform-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>fit and transform</a></span></li></ul></li><li><span><a href="#implicit" data-toc-modified-id="implicit-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>implicit</a></span><ul class="toc-item"><li><span><a href="#prepare" data-toc-modified-id="prepare-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>prepare</a></span></li><li><span><a href="#here-we-start" data-toc-modified-id="here-we-start-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span><font color="red">here we start</font></a></span></li></ul></li></ul></div>

# FeatureProducer.py

In [1]:
from lib.FeatureProducer import FeatureProducer

# main

## init

In [2]:
fp = FeatureProducer(root='../data')

## load

In [3]:
fp.load_raw()

15:10:30 INFO << load raw data in 19.99s


## preprocess

In [4]:
fp.pre_process()

15:10:59 INFO << parse train_df in 29.67s
15:11:16 INFO << parse test_df in 16.56s
15:11:17 INFO << parse members_df in 0.93s
15:11:34 INFO << parse songs_df in 16.45s
15:11:37 INFO << parse song_extra_info_df in 3.02s


## feature engineering

In [5]:
fp.feature_engineering()

15:12:31 INFO << parse engineering_df in 26.49s
15:12:52 INFO << parse engineering_df in 21.43s


## fit and transform

In [7]:
fp.self_fit_transform()

15:14:00 INFO << transform "msno" in 2.27s
15:14:04 INFO << transform "song_id" in 4.07s
15:14:06 INFO << transform "source_system_tab" in 1.32s
15:14:07 INFO << transform "source_screen_name" in 1.38s
15:14:08 INFO << transform "source_type" in 1.28s
15:14:09 INFO << transform "1h_source" in 0.41s
15:14:09 INFO << transform "1h_system_tab" in 0.38s
15:14:10 INFO << transform "1h_screen_name" in 0.35s
15:14:10 INFO << transform "1h_source_type" in 0.33s
15:14:10 INFO << transform "song_length" in 0.33s
15:14:11 INFO << transform "genre_ids" in 1.31s
15:14:13 INFO << transform "artist_name" in 1.78s
15:14:15 INFO << transform "composer" in 1.72s
15:14:16 INFO << transform "lyricist" in 1.10s
15:14:17 INFO << transform "language" in 1.09s
15:14:17 INFO << transform "artist_count" in 0.29s
15:14:18 INFO << transform "genre_count" in 0.27s
15:14:18 INFO << transform "composer_count" in 0.24s
15:14:18 INFO << transform "lyricist_count" in 0.21s
15:14:18 INFO << transform "1h_lang" in 0.16s


# implicit

## prepare

In [9]:
train_df = fp.train_df
test_df = fp.test_df
comb_df = fp.comb_df
songs_df = fp.songs_df
song_extra_info_df = fp.song_extra_info_df
members_df = fp.members_df

train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377418 entries, 0 to 7377417
Data columns (total 38 columns):
msno                       category
song_id                    category
source_system_tab          category
source_screen_name         category
source_type                category
target                     int64
1h_source                  bool
1h_system_tab              bool
1h_screen_name             bool
1h_source_type             bool
song_length                int32
genre_ids                  category
artist_name                category
composer                   category
lyricist                   category
language                   category
artist_count               int32
genre_count                int32
composer_count             int32
lyricist_count             int32
1h_lang                    bool
1h_song_length             bool
song_year                  int32
1h_song_year               bool
city                       category
bd                         category


In [10]:
train_df.head(10)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,1h_source,1h_system_tab,1h_screen_name,1h_source_type,...,expiration_date,membership_days,registration_init_year,registration_init_month,expiration_date_year,expiration_date_month,1h_via,play_count,track_count,cover_lang
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,False,False,False,False,...,20171005,2103,2012,1,2017,10,True,215,84,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,True,True,True,True,...,20170911,2301,2011,5,2017,9,True,1,17080,10
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,True,True,True,True,...,20170911,2301,2011,5,2017,9,True,4,61,2
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,True,True,True,True,...,20170911,2301,2011,5,2017,9,True,1,1,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,False,False,False,False,...,20171005,2103,2012,1,2017,10,True,412,6,1
5,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,explore,Explore,online-playlist,1,False,False,False,False,...,20171005,2103,2012,1,2017,10,True,1108,15,1
6,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,VkILU0H1h3NMmk9MQrXouNudGk5n8Ls5cqRRuBxeTh4=,my library,Local playlist more,local-playlist,1,True,True,True,True,...,20170911,2301,2011,5,2017,9,True,3869,43,1
7,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,bPIvRTzfHxH5LgHrStll+tYwSQNVV8PySgA3M1PfTgc=,explore,Explore,online-playlist,1,False,False,False,False,...,20171005,2103,2012,1,2017,10,True,287,33,1
8,uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE=,/bU6IRSK+YNlNbaTkxo7bhsb2EDLPrnksdX3ggcZNhI=,my library,Local playlist more,local-library,1,True,True,True,True,...,20180304,2309,2011,11,2018,3,True,31,155,2
9,uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE=,EbI7xoNxI+3QSsiHxL13zBdgHIJOwa3srHd7cDcnJ0g=,my library,Local playlist more,local-library,1,True,True,True,True,...,20180304,2309,2011,11,2018,3,True,738,155,2


## <font color='red'>here we start</font>